In [23]:
from flask import jsonify
import requests
import datetime
import pandas as pd
from model.LSTMModel import LSTMModel
from model.RNNModel import RNNModel
from model.XgboostModel import XGBoostModel

def fetch_binance_data(symbol="BTCUSDT", interval="5m", limit=200):
    url = f"https://api.binance.us/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        # get last row of data
        data = response.json()
        df = pd.DataFrame(data, columns=['t', 'o', 'h', 'l', 'c', 'v', 'T', 'q', 'n', 'V', 'Q', 'B'])
        df = df[['t', 'o', 'h', 'l', 'c']]
        df['t'] = pd.to_datetime(df['t'], unit='ms')
        print(df)
        
        # Load model
        model = XGBoostModel(symbol)
        model.load('./trained/BTC-XGBoost_Close')

        if model:
            # Ensure column names match the expected names used in the model prediction
            df.rename(columns={'c': 'Close'}, inplace=True)
            df.rename(columns={'t': 'Date'}, inplace=True)
            df.rename(columns={'o': 'Open'}, inplace=True)
            df.rename(columns={'h': 'High'}, inplace=True)
            df.rename(columns={'l': 'Low'}, inplace=True)
            
            predictions = model.predict(df[['Date','Open','Close','High', 'Low']])
            print(f' prediction: {predictions}')
            # Format predictions as required ([[t, o, h, l, c], [t, o, h, l, c], ...])
            prediction_list = []
            for prediction in predictions:
                prediction_list.append([ prediction['Open'], prediction['High'], prediction['Low'], prediction['Close']])

            print(f' Prediction list: {prediction_list}')
           
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Example usage
data = fetch_binance_data()

                      t               o               h               l  \
0   2024-07-15 07:25:00  62784.08000000  62788.40000000  62748.29000000   
1   2024-07-15 07:30:00  62818.00000000  62861.50000000  62818.00000000   
2   2024-07-15 07:35:00  62868.65000000  62917.40000000  62783.65000000   
3   2024-07-15 07:40:00  62919.00000000  63050.25000000  62914.10000000   
4   2024-07-15 07:45:00  62870.09000000  62870.09000000  62819.59000000   
..                  ...             ...             ...             ...   
195 2024-07-15 23:40:00  64550.00000000  64550.00000000  64526.69000000   
196 2024-07-15 23:45:00  64500.04000000  64550.00000000  64500.03000000   
197 2024-07-15 23:50:00  64518.79000000  64651.38000000  64500.00000000   
198 2024-07-15 23:55:00  64651.38000000  64709.17000000  64587.08000000   
199 2024-07-16 00:00:00  64682.70000000  64699.49000000  64585.60000000   

                  c  
0    62788.40000000  
1    62840.60000000  
2    62917.40000000  
3    62982.

c:\Users\Admin\OneDrive - VNU-HCMUS\Nam4\ModernTech\Stock-Prediction-Dashboard\.venv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
 prediction: [[62649.797 64512.707 61697.195 63481.09 ]
 [62779.2   64660.227 61832.207 63601.59 ]
 [62814.83  64668.426 61855.86  63630.7  ]
 [62858.688 64720.773 61906.918 63668.035]
 [62780.164 64643.016 61818.42  63601.84 ]
 [62750.625 64626.547 61803.777 63571.902]
 [62706.562 64577.03  61745.72  63536.875]
 [62737.62  64612.664 61786.38  63546.16 ]
 [62819.184 64660.777 61854.55  63620.42 ]
 [62798.73  64670.965 61847.324 63612.7  ]
 [62826.613 64693.766 61871.926 63650.074]
 [62808.71  64686.98  61863.863 63632.887]
 [62894.07  64760.914 61937.43  63701.88 ]
 [62929.76  64794.207 61979.78  63737.55 ]
 [62647.45  64526.453 61684.11  63470.19 ]
 [62833.254 64705.254 61878.945 63648.39 ]
 [62958.285 64805.875 62003.23  63757.14 ]
 [62960.6   64817.156 61998.58  63768.81 ]
 [63019.902 64878.684 62080.48  63839.33 ]
 [62993.047 64860.234 62050.246 63801.824]
 [63105.473 64979.54  62143.684 63937.23 ]
 [62933.105 64810.83  61990.125 63754.89 ]
 [6

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices